In [15]:
#Import library
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
import os
import TextProcess.text_process as text_process

In [9]:
def read_input(file_name):
    """
    read input file which is tsv file
    
    """
    df =  pd.read_csv(os.path.join('./data',file_name), sep="\t")
    return df

In [4]:
#Import dataset
train=read_input('train.tsv')
train.head(3)

,Unnamed: 0,product_id,description2,title,label
0,0,100191,JACK OF CLUBS PSA 5 9 OF SPADES PSA 5 7 OF HEA...,(4) 1889 N220 KINNEY HARLEQUIN PSA 5 EX TOBACC...,0
1,1,100217,Powered by Frooition About us Newsletter Feedb...,12 Jars Of Dalfour Beauty Gold Seal EXCEL Beau...,0
2,2,100272,StrawberryNET Check out our eBay store >> Cate...,Perricone MD Vitamin C Ester Eye Serum 15ml/0.5oz,0


In [35]:
train['text_label']=train['label'].apply(lambda x: "Non Violation" if x == 1 else "Violation " )

In [37]:
test['text_label']=test['label'].apply(lambda x: "Non Violation" if x == 1 else "Violation " )

In [10]:
test=read_input('dev.tsv')
test.head(3)

,Unnamed: 0,product_id,description2,title,label
0,0,100394,TOP Gold Light 100's 100MM - 1 Box - 200 Tubes...,TOP Gold Light 100's 100MM - 1 Box - 200 Tubes...,0
1,1,100692,15oceaneshop Add to my favorite sellers Mon. t...,Portable Clear Mini Acrylic Water Pipes Smokin...,0
2,2,100857,StrawberryNET Check out our eBay store >> Cate...,Clarins Daily Energizer Cleansing Gel 75ml/2.5oz,0


In [16]:
def preprocess_text(lists):
    """
    Remove noise, normalize and tokenize text
    """
    preprocess_text_list=[]
    for i in range(len(lists)):
        if lists[i]=='None':
            preprocess_text_list.append('None')
        else:
            try:
                preprocess_text_list.append(text_process.text_processing(lists[i]))
            except:
                preprocess_text_list.append('None')
    return (preprocess_text_list)

In [17]:
#Tokenization
train_description_tokenize=preprocess_text(train['description2']) #Tokenize description of train set
train_title_tokenize=preprocess_text(train['title'])  #Tokenize title of train set
test_description_tokenize=preprocess_text(test['description2'])  #Tokenize description of test set
test_title_tokenize=preprocess_text(test['title']) #Tokenize title of test set

In [18]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [56]:
des_train_tagged = [TaggedDocument(words=train_description_tokenize[i], tags=[train['text_label'][i]]) for i in range(len(train_description_tokenize))]
des_test_tagged = [TaggedDocument(words=test_description_tokenize[i], tags=[test['text_label'][i]]) for i in range(len(test_description_tokenize))]


In [57]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])



C:\Users\defaultuser0.LAPTOP-6F0LJR1V\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
100%|█████████████████████████████████████████████████████████████████████████| 3107/3107 [00:00<00:00, 1039127.86it/s]


KeyboardInterrupt: 

In [ ]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors
model_doc2vec.save('./movieModel.d2v')

In [54]:
#Building a Vocabulary
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(des_train_tagged)])

C:\Users\defaultuser0.LAPTOP-6F0LJR1V\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
100%|█████████████████████████████████████████████████████████████████████████| 3107/3107 [00:00<00:00, 1557139.75it/s]


In [53]:
des_test_tagged[30]

TaggedDocument(words=['bid', 'antique', 'pack', 'sweet', 'mist', 'cigarette', 'tobacco', 'open', 'still', 'seal'], tags=['Non Violation'])

In [47]:
des_test_tagged=np.array(des_test_tagged)

In [55]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(des_train_tagged)]), total_examples=len(des_train_tagged), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha
    print(epoch)

100%|██████████████████████████████████████████████████████████████████████████| 3107/3107 [00:00<00:00, 778802.52it/s]


KeyboardInterrupt: 